In [13]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow 
from mlflow.models import infer_signature

In [18]:
#load the dataset
url = "https://raw.githubusercontent.com/dsrscientist/DSData/master/winequality-red.csv"
df = pd.read_csv(url)
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [19]:
#split to train test data 
train,test=train_test_split(df,test_size=0.25,random_state=42)



In [20]:
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
582,11.7,0.490,0.49,2.2,0.083,5.0,15.0,1.00000,3.19,0.43,9.2,5
626,8.8,0.600,0.29,2.2,0.098,5.0,15.0,0.99880,3.36,0.49,9.1,5
1030,7.1,0.590,0.00,2.1,0.091,9.0,14.0,0.99488,3.42,0.55,11.5,7
620,8.3,0.540,0.24,3.4,0.076,16.0,112.0,0.99760,3.27,0.61,9.4,5
490,9.3,0.775,0.27,2.8,0.078,24.0,56.0,0.99840,3.31,0.67,10.6,6
...,...,...,...,...,...,...,...,...,...,...,...,...
1130,9.1,0.600,0.00,1.9,0.058,5.0,10.0,0.99770,3.18,0.63,10.4,6
1294,8.2,0.635,0.10,2.1,0.073,25.0,60.0,0.99638,3.29,0.75,10.9,6
860,7.2,0.620,0.06,2.7,0.077,15.0,85.0,0.99746,3.51,0.54,9.5,5
1459,7.9,0.200,0.35,1.7,0.054,7.0,15.0,0.99458,3.32,0.80,11.9,7


In [22]:
train[['quality']].values

array([[5],
       [5],
       [7],
       ...,
       [5],
       [7],
       [6]], dtype=int64)

In [21]:
train[['quality']].values.ravel()#convert te 2d to 1d

array([5, 5, 7, ..., 5, 7, 6], dtype=int64)

In [23]:
train_x=train.drop(['quality'],axis=1).values
test_x=train.drop(['quality'],axis=1).values
train_y=train[['quality']].values.ravel()
test_y=test[['quality']].values.ravel()

In [24]:
#split data to train and validata data 
train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.2,random_state=42)


In [25]:
signature=infer_signature(train_x,train_y)

In [27]:
mean=np.mean(train_x,axis=0)#mean col wise
mean

array([ 8.29781022,  0.53131387,  0.2712513 ,  2.52591241,  0.08946403,
       15.88008342, 47.07351408,  0.99671879,  3.30930136,  0.6632951 ,
       10.42846715])

In [30]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")
import os
os.environ['GIT_PYTHON_GIT_EXECUTABLE'] = r'C:\Program Files\Git\cmd\git.exe'  # Adjust the path if needed

In [47]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)

        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params wwith MLFLOW tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
                  epochs=epochs,
                  batch_size=64)
        
        ## Evaluate the model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [48]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [49]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)

}

In [54]:
mlflow.set_experiment("wine")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


2025/02/04 17:13:18 INFO mlflow.tracking.fluent: Experiment with name 'wine' does not exist. Creating a new experiment.


Epoch 1/3                                            

15/15 [==============================] - 1s 17ms/step - loss: 5.2216 - root_mean_squared_error: 2.2851 - val_loss: 0.6623 - val_root_mean_squared_error: 0.8138

Epoch 2/3                                            

15/15 [==============================] - 0s 6ms/step - loss: 0.5679 - root_mean_squared_error: 0.7536 - val_loss: 0.5167 - val_root_mean_squared_error: 0.7188

Epoch 3/3                                            

15/15 [==============================] - 0s 6ms/step - loss: 0.4291 - root_mean_squared_error: 0.6551 - val_loss: 0.5233 - val_root_mean_squared_error: 0.7234

4/4 [==============================] - 0s 4ms/step - loss: 0.5233 - root_mean_squared_error: 0.7234

  0%|          | 0/4 [00:01<?, ?trial/s, best loss=?]INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmpf3cjp2e2\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmpf3cjp2e2\model\data\model\assets



Epoch 1/3                                                                      

15/15 [==============================] - 1s 12ms/step - loss: 6.2461 - root_mean_squared_error: 2.4992 - val_loss: 1.3419 - val_root_mean_squared_error: 1.1584

Epoch 2/3                                                                      

15/15 [==============================] - 0s 5ms/step - loss: 1.1762 - root_mean_squared_error: 1.0845 - val_loss: 1.0537 - val_root_mean_squared_error: 1.0265

Epoch 3/3                                                                      

15/15 [==============================] - 0s 4ms/step - loss: 0.9143 - root_mean_squared_error: 0.9562 - val_loss: 0.9031 - val_root_mean_squared_error: 0.9503

4/4 [==============================] - 0s 4ms/step - loss: 0.9031 - root_mean_squared_error: 0.9503

 25%|██▌       | 1/4 [00:09<00:26,  8.69s/trial, best loss: 0.7233788371086121]INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmp_y7t51az\model\data\model\a

INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmp_y7t51az\model\data\model\assets



...

Epoch 1/3                                                                      

15/15 [==============================] - 1s 14ms/step - loss: 6.2748 - root_mean_squared_error: 2.5050 - val_loss: 0.8289 - val_root_mean_squared_error: 0.9104

Epoch 2/3                                                                      

15/15 [==============================] - 0s 6ms/step - loss: 0.5661 - root_mean_squared_error: 0.7524 - val_loss: 0.5935 - val_root_mean_squared_error: 0.7704

Epoch 3/3                                                                      

15/15 [==============================] - 0s 5ms/step - loss: 0.4783 - root_mean_squared_error: 0.6916 - val_loss: 0.7494 - val_root_mean_squared_error: 0.8657

4/4 [==============================] - 0s 4ms/step - loss: 0.7494 - root_mean_squared_error: 0.8657

 75%|███████▌  | 3/4 [00:26<00:08,  8.58s/trial, best loss: 0.7233788371086121]INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmpzjd9vec5\model\data\model\a

INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmpzjd9vec5\model\data\model\assets



100%|██████████| 4/4 [00:34<00:00,  8.73s/trial, best loss: 0.7233788371086121]
INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmpvangcjpn\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\hp\AppData\Local\Temp\tmpvangcjpn\model\data\model\assets



Best parameters: {'lr': 0.07783557287550716, 'momentum': 0.7162173115007349}
Best eval rmse: 0.7233788371086121


In [60]:

#inferencing 
import mlflow
from mlflow.models import validate_serving_input
model_uri = 'runs:/07f17843f2d64d718cf83dd1ea8a6a91/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

c:\Users\hp\anaconda3\envs\mlflow_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


38/38 [==============================] - 0s 2ms/step


array([[4.8550124],
       [4.999224 ],
       [6.1058187],
       ...,
       [5.2245913],
       [7.058051 ],
       [6.9983563]], dtype=float32)

In [61]:
# Load model as a PyFuncModel. other way to infer the model 
model_uri = 'runs:/07f17843f2d64d718cf83dd1ea8a6a91/model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

38/38 [==============================] - 0s 2ms/step


array([[4.8550124],
       [4.999224 ],
       [6.1058187],
       ...,
       [5.2245913],
       [7.058051 ],
       [6.9983563]], dtype=float32)

In [62]:
## Register in the model registry using code
mlflow.register_model(model_uri,"wine-quality")

Successfully registered model 'wine-quality'.
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1738669801117, current_stage='None', description=None, last_updated_timestamp=1738669801117, name='wine-quality', run_id='07f17843f2d64d718cf83dd1ea8a6a91', run_link=None, source='file:///d:/projects/mlflow%20wine%20with%20hyperops%20ann/mlruns/113578519555664419/07f17843f2d64d718cf83dd1ea8a6a91/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>